# Classification of valve condition into TAV, BAV, MAV, and AS

## Import necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks as CB
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Rescaling 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import add
from sklearn.metrics import roc_auc_score
import numpy as np
import os
import tensorflow.keras as keras
import scipy.io
import h5py
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# specify the gpu to be utilized

## Create the multi-layer perceptron (MLP) and convolutional neural network (CNN) models

The input of the MLP is the demographic information of the subjects, including their weight, height, age, and gender. The MLLP model is composed of two fully-connected dense layers. All three continuous attributes are min-max normalized, and the participants' gender are one-hot encoded (i.e. female = 1, male = 0) before being fed into the MLP.  
    
The input of the CNN model is the scalograms of the recorded SCG pulses. The CNN architecture consisted of a stack of five convolutional blocks, with each block containing a sequence of Conv2D, ReLU (rectified linear unit) activation, batch normalization, and max pooling layers. 

In [ ]:
# MLP model
def create_mlp(dim):
    # define the Multilayer Perceptron (MLP) network:
    model = Sequential()
    model.add(Dense(8,input_dim=dim,activation="relu"))
    model.add(Dense(4,activation="relu"))
    # return the model
    return model

# CNN model
def create_cnn(image_size=(256,256,1),filters=(64,128,256,512),augmentation=False):
    # define the model input
    inputs = Input(shape=image_size)
    # data augmentation
    if augmentation:
        x = RandomFlip("horizontal")(inputs)
        x = RandomRotation(0.1)(x)
        x = RandomZoom(0.2)(x)
    # loop over filters
    for i, filter in enumerate(filters):
        # if this is the first layer, set the input appropriately
        if i == 0 and not augmentation:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(filter,kernel_size=3,padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=2)(x)
    # flatten the volume, then FC=>RELU=>BN=>DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.5)(x)
    # apply anothe layer of FC layer, this one matches the number of nodes
    # coming out of the MLP 
    x = Dense(4)(x)
    outputs = Activation("relu")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    # return the CNN model
    return model

## Define a function to load the .mat files 

The .mat files contain three type of information:

    The "encAsValve" contains the one-hot encoded aortic valve condition classes
    The "attr" contains the demographic attributes of the subjects
    The "scalogram" contains the scalograms of the seismocardiogram pulses

In [ ]:
def load_mat_file(mat_file):
    f = h5py.File(mat_file+'.mat','r')
    variables = f.items()
    for var in variables:
        name = var[0]
        data = var[1]
        if type(data) is not h5py.Dataset:
            continue
        if str(name)=="encAsValve":
            encAsValve = np.array(data,dtype=np.float32)
            encAsValve = np.transpose(encAsValve,(1,0))
        elif str(name)=="attr":
            attr = np.array(data,dtype=np.float32)
            attr = np.transpose(attr,(1,0))
        elif str(name)=="scalogram":
            scg = np.array(data,dtype=np.float32)
            scg = np.transpose(scg,(2,1,0))
            sh  = np.shape(scg)
            scg = np.resize(scg,(sh[0],sh[1],sh[2],1))
    return scg, attr, encAsValve

## Run the model for multiple iterations - leave-subject-out cross validation

To evaluate the performance of the models, we use a leave-subject-out cross-validation approach. This means that for each iteration, we use 50% of the available SCG pulses (N = 6249) for training the model and reserve the remaining 50% for testing. Importantly, the SCG pulses belonging to each subject are only included in either the training or test set, but not both, in order to avoid any potential bias in the performance metrics. We repeate this process for 10 iterations, randomly selecting different subjects to be included in the training and test sets at each trial. This allows us to verify that the reported performance metrics are not influenced by any specific patterns in the distribution of the training and test sets. For each iteration, the model is trained for 150 epochs. 

In [ ]:
# define a callback function to monitor the model's performance over the validation set
callback = CB.ModelCheckpoint("mixed_mdl_class.keras",
                             save_best_only=True, 
                              monitor="val_loss")
# tensorboard = CB.TensorBoard(log_dir='/home/mem1342/Projects/quest_scg_regression')
history = list()
for iterations in range(1,11):
    # read the training set
    scg_train,atr_train,encAsValve_train=load_mat_file(os.path.join('scg_dataset/50-50',
                                                                     'train_cv_'+str(iterations)))
    # return the predictors of the training set
    train_x = [atr_train,scg_train]
    # return the labels of training set
    train_y = encAsValve_train
    # read the test set
    scg_test,atr_test,encAsValve_test=load_mat_file(os.path.join('scg_dataset/50-50',
                                                                  'test_cv_'+str(iterations)))
    # return the predictors of test set
    test_x = [atr_test,scg_test]
    # return the labels of test set n
    test_y = encAsValve_test
    # create a multi-layer perceptron model based on subject attributes
    mlp = create_mlp(atr_train.shape[1])
    # create a cnn model based on scalograms
    cnn = create_cnn(image_size=(256,256,1),filters=[4,8,16,32,128],augmentation=False)
    # combine created mlp and cnn models
    combInp = keras.layers.concatenate([mlp.output,cnn.output])
    x = keras.layers.Dense(4,activation="relu")(combInp)
    # return the output associated with combination of aortic stenosis and the valve condition
    y1 = keras.layers.Dense(4,activation="softmax",name='aortic_stenosis_valve')(x)
    model = keras.Model(inputs=[mlp.input,cnn.input],outputs=y1)
    # compile the mixture model using mean absolute percentage error as loss function
    opt = Adam(learning_rate=1e-3, decay=1e-3 / 100)
    model.compile(optimizer=opt, 
              loss={'aortic_stenosis_valve': 'categorical_crossentropy'},
              metrics={'aortic_stenosis_valve': 'accuracy'})
    # train the model
    hist = model.fit(x=train_x, 
                  y=train_y,
                  validation_data = (test_x, test_y),
                  epochs=150,
                  batch_size=32, 
                  verbose=2,
                  callbacks=[callback])
    history.append(hist)
    # load the model at the epoch with the lowest error over the validation set
    model = tf.keras.models.load_model("mixed_mdl_class.keras")
    # predict the vmax in test set
    preds = model.predict(test_x,verbose=2)
    # return the true and predicted aortic stenosis condition
    pred = np.array(preds)
    asValve = list() 
    asValve.append(encAsValve_test)
    asValve.append(pred)
    filename1 = os.path.join('/home/mem1342/Projects/quest_scg_regression/results',
                             'cv_'+str(iterations)+'.mat')
    scipy.io.savemat(filename1,{'as_valve':asValve})
    # save the roc curves
    roc_auc = np.zeros(shape=(4,))
    for i in range(4):
        roc_auc[i] = roc_auc_score(encAsValve_test[:,i],pred[:,i])
    print(f"The mean ROC AUC value: {np.mean(roc_auc): 0.2f} ")
    tr_acc = hist.history["accuracy"]
    val_acc = hist.history["val_accuracy"]
    epochs = range(1,len(tr_acc)+1)
    plt.figure()
    plt.plot(epochs,tr_acc,'bo',label="Training")
    plt.plot(epochs,val_acc,'ro',label="Validation")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f"The mean ROC AUC value: {np.mean(roc_auc): 0.2f} ")
    plt.legend()
    plt.show()